# Simulering av tilfeldige tårn

Innføring i sjakkoding med Python finner du på [bit.ly/sjakkmatte](bit.ly/sjakkmatte).

Sjakkommandoene vi bruker i denne oppgaven er:

|  Kommando |  Forklaring |
|---|---|
|  `1 = Board(None)` | Opprett nytt tomt sjakkbrett  |
| `1`  |  Vis brett 1 ved kjøring av programmet |
|  `1.set_piece_at(2,3)` |  På brett 1: Sett brikke 3 på felt 2 |
|  `1 = random_rooks()` | Opprett nytt sjakkbrett med tilfeldig plassering av svarte tårn |
|  `show_paths(1)` | På brett 1: Viser eventuelle trygge kongeveier |
|  `who_has_path(1)` | På brett 1: Spør om hvem som har trygg kongevei - for eksempel betyr resultatet `[True, False]` at hvit har trygg kongevei, men ikke svart |

Tallene skal erstattes med

1. Et variabelnavn for sjakkbrettet
2. Felt: `A1`, `A2`, ..., `H8`, eller skriv tall mellom `0` og `63`
3. Brikketype: `r` for svart tårn

Kommandoen `random_rooks()` lages selv i den første oppgaven, mens resten av kommandoene importeres ved å kjøre den neste kodeblokken.
Det er ikke meningen at denne koden skal forstås, men kommentarer er lagt til for spesielt interesserte. 
For å kjøre koden klikker du på kodeblokken og bruker tastekombinasjonen `Ctrl + Enter`:

In [ ]:
from random import randint
from chess import *
from chess.svg import board as show_board
r = Piece(4, 0) #black rook
def diff(l1, l2):
    return [x for x in l1 if x not in l2] 

#we will refer to the Hex theorem, which states that the only possible outcomes are
    #white has path, but black has not
    #black has path, but white has not
#path = safe king path

#class to find safe king paths when placement of rooks is given (as set of squares)
#squares are referenced by numbers according to the figure further down
class KingPaths:
    def __init__(self, rooks):
        self.rooks = set(rooks)
        self.num_rooks = len(rooks)
        self.empty = set(range(0,64)) - self.rooks #empty squares 
        self.white_starts = set(range(56,64)) & self.empty
        self.black_starts = set(range(0,57,8)) & self.rooks
        self.white_goals = set(range(0,8)) & self.empty
        self.black_goals = set(range(7,64,8)) & self.rooks
        
        self.legal_moves = [None]*64  #legal moves from each square
        #(legal white moves from empty squares, legal black moves from rook squares)
        
        self.min_size = 64 #size of smallest path
        self.path = [] #found path for either white or black (sequence of squares)
        self.path_found = False #whether path has been found or not
        self.white_has_path = None #True if path belongs to white, False if path belongs to black
    
    #get either empty squares (for white) or rook squares (for black)
    def get_squares(self, white):
        return self.empty if white else self.rooks
    
    #get starting squares for either white or black
    def get_starts(self, white):
        return self.white_starts if white else self.black_starts
    
    #get goal squares for either white or black
    def get_goals(self, white):
        return self.white_goals if white else self.black_goals
        
    #get legal moves from given square, for eiter white or black
    def get_legal_moves(self, white, square):
        km = set(SquareSet(BB_KING_ATTACKS[square])) #king moves from square
        
        #forbidden squares for both white and black side due to "king injury"
        ne = {square+9} if (square+9 < 64) else set() #northeast
        sw = {square-9} if (square-9 > -1) else set() #southwest
        
        #forbidden squares for either white or black
        #white must avoid rook squares and black must avoid empty squares
        #we also don't want to go back to starting squares (waste of time)
        fs = self.rooks | self.white_starts if white else self.empty | self.black_starts 
        
        #legal moves are king moves minus forbidden squares
        move_set = km - ne - sw - fs
        
        #sort moves according to what's most likely on a path:
        if white:
            moves = list(move_set) 
            moves.sort() #put down moves first
        else: 
            right_moves = {square + 1, square-7}
            moves = list(move_set & right_moves) + list(move_set - right_moves) #put right moves first
            
        return moves
    
    #calculate and save legal moves from all empty squares (for white), or from all rook squares (for black)
    def set_legal_moves(self, white):
        for x in self.get_squares(white):
            self.legal_moves[x] = self.get_legal_moves(white, x)
    
    #recursive function to walk to next legal square
    def walk(self, find_smallest, white, path, size):
        #if path has been found and we don't need smallest path:
        if self.path_found and (not find_smallest): 
            return
        #if we have already found a complete path of smaller size than current partial path:
        if self.path_found and (size > self.min_size-1):
            return
        
        current = path[-1] #current square
        
        #if we have reached one of the goal squares:
        if current in self.get_goals(white):
            self.path_found = True
            self.min_size = size
            self.path = path.copy() #important to make a copy, since path variable will change later
            return
        
        #we have not reached a goal square, so we try extending the current path
        #we first make a list of legal squares from the current square
        #we also avoid previous squares on the path
        legal_moves =  diff(self.legal_moves[current], path)
        
        #try walking to next legal squares
        for x in legal_moves:
            path.append(x)
            self.walk(find_smallest, white, path, size + 1)
            path.pop(-1)
        
            #alternative to these three lines is the following line: 
            #self.walk(find_smallest, white, path + [x], size + 1)
            #path + [x] is then a new list for each move
            #we have instead chosen to reuse the list stored in variable 'path', saving memory
            
        return
    
    #search for a path for either white or black
    def partial_search(self, find_smallest, white):
        self.set_legal_moves(white) #we calculate and save the legal moves from all squares that we might walk on
        #it's useful to save this information before we start walking, 
        #because squares might be visited multiple times,
        #and we don't want to calculate the legal moves over again on the second visit
        
        #we start walking from each starting square
        for x in self.get_starts(white):
            self.walk(find_smallest, white, [x], 1)
    
    #search for any path
    def full_search(self, find_path, find_smallest):
        #we begin searching for the side that is most likely to have a path, based on the number of rooks:
        white = self.num_rooks < 33   
        self.partial_search(find_smallest, white)
        
        #by the Hex theorem, searching one side is always enough to establish who has path
        self.white_has_path = white if self.path_found else not white
        
        #if we don't need to know actual path, but only establish who has path, we are now done
        if not find_path:
            return
        
        #if we want the actual path, and it was not found for the first side, we search the second side:
        if not self.path_found:
            self.partial_search(find_smallest, not white)
        
        #a path has now been found be the Hex theorem
        return
    
    def get_smallest_path(self):
        self.full_search(True, True) #(True, True) because we want the path, and the smallest one
        return self.path 
    
    def get_white_has_path(self):
        self.full_search(False, False) #(False, False) because we don't need a path (and therefore not smallest either)
        return self.white_has_path

#functions described in the table:

def show_paths(board):
    rooks = set(board.pieces(4,0)) #get the squares occupied by black rooks, as set of numbers
    kp = KingPaths(rooks) 
    path = kp.get_smallest_path() #smallest path as list of numbers
    
    #print a message about who has a path:
    if kp.white_has_path:
        print("Hvit har en trygg kongevei!")
    else:
        print("Svart har en trygg kongevei!")
        
    #return an svg figure of the board with the smallest path marked
    return show_board(board, squares=SquareSet(path), size=390) 

def who_has_path(board):
    rooks = set(board.pieces(4,0))
    white_has_path = KingPaths(rooks).get_white_has_path()
    
    #by Hex theorem, the two values are always opposite
    return [white_has_path, not white_has_path]

## Oppgave 1: Simuleringsfunksjon

For å simulere tilfeldige forsøk, trenger vi en funksjon som gir oss tilfeldige tall. 
Til det bruker vi `randint(0,1)`, som plukker tilfeldig $0$ eller $1$, med $50\%$ sannsynlighet for hver. 
Dette simulerer altså et myntkast, siden vi for eksempel kan si at $0$ betyr kron og $1$ betyr mynt. 

**(a)** Kjør koden under flere ganger og sjekk at resultatet er tilfeldig. Hvis du ønsker ekstra trening kan du prøve å skrive ut en melding, for eksempel "det ble kron" hvis resultatet ble $0$, og "det ble mynt" hvis resultatet ble $1$. 

In [ ]:
tilfeldig_tall = randint(0,1)
print(tilfeldig_tall)

Nå skal du opprette et sjakkbrett, og simulere forsøket der vi plasserer svarte tårn tilfeldig på brettet. For hvert felt skal det være 50% sannsynlighet for å sette på tårn (og derfor 50% sannsynlighet for å la det stå tomt). Husk at feltene på sjakkbrettet er nummerert mellom 0-63, som vist på figuren:

<img src="sjakkbrett.png" width="300"/>


**(b)** Vi har definert funksjonen `random_rooks()` i kodeblokken under. Nå returneres bare et tomt brett, men du skal sørge for at brettet har svarte tårn på tilfeldige felter. Husk at du finner kommandoen for å sette på brikke i tabellen øverst.

In [ ]:
def random_rooks():
    brett = Board(None)
    #skriv kode her:
    
    
    
    
    return brett

testbrett = random_rooks()
testbrett

## Oppgave 2: Relativ frekvens for hånd
Vi antar videre at funksjonen `random_rooks()` fungerer. Hvis du ikke fikk til forrige oppgave, må du kjøre koden under for å få funksjonen til å fungere.

In [ ]:
def random_rooks():
    brett = Board(None)
    for i in range(64):
        if randint(0,1):
            brett.set_piece_at(i, r)
    return brett

Vi skal nå simulere forsøket mange ganger, og hver gang skal vi sjekke om hvit eller svart har en trygg kongerute. 

|Nr|Hvit|Svart|
|--|--|--|
|1|x||
|2||x|

I dette eksempelet hadde hvit en trygg kongevei i første simulering, og svart hadde en trygg kongevei i andre simulering. Skriv opp din egen tabell på papir med de samme kolonnene.

**(a)** Kjør koden under mange ganger, og legg til en ny rad i tabellen din hver gang. Kryss av om hvit og/eller svart har en trygg kongevei. 

**(b)** Når du har mange rader i tabellen,  regn ut den relative frekvensen for følgende hendelser:
* $H$ = Hvit har trygg kongevei
* $S$ = Svart har trygg kongevei


In [ ]:
brett = random_rooks()
show_paths(brett)

## Oppgave 3: Relativ frekvens med programmering

Når du har opprettet et tilfeldig tårnbrett med kodelinjen `brett = random_rooks()`, kan du bruke funksjonen `who_has_path(brett)` til å teste hvem som har en trygg kongevei. 
Funksjonen gir en liste som svar.
Hvis du for eksempel får svaret `[True, False]` betyr det at hvit har en trygg kongevei (siden første element er `True`), men svart har ikke en trygg kongevei (siden andre element er `False`). 

**(a)** Skriv et program som simulerer 500 tilfeldige tårnbrett, og regner ut den relative frekvensen for hendelsene:  
* $H$ = Hvit har trygg kongevei
* $S$ = Svart har trygg kongevei

Vær oppmerksom på at koden kan ta noen sekunder å kjøre med flere hundre simuleringer.

In [ ]:
#skriv kode her:





## Oppgave 4: Sannsynlighet og komplementære hendelser

Et utfall i vårt forsøk er et "tårnbrett", altså et sjakkbrett med svarte tårn på noen av feltene (det er også mulig at alle felter får tårn, eller at alle felter blir tomme).
* Hendelsen $H$ består av alle tårnbrett der hvit har en trygg kongevei. 
* Hendeslen $S$ består av alle tårnbrett der svart har en trygg kongevei. 

**(a)** Bruk kodeblokken under til å simulere forsøket mange ganger. Ved hver simulering skal du skrive ut resultatet av kommandoen `who_has_path()`.

In [ ]:
#skriv kode her:





Når du har gjort mange simuleringer, svar på følgende spørsmål.

**(a1)** Tror du det finnes tårnbrett som hverken er i $H$ eller $S$ (det vil si at ingen sider har trygg kongevei)?

**(a2)** Tror du det finnes tårnbrett som både er i $H$ of $S$ (det vil si at begge sider har trygg kongevei)? 
    
**(b)** Tror du at $H$ og $S$ er komplementære? Det betyr at alle tårnbrett finnes i en av hendelsene, men ingen tårnbrett finnes i begge hendelsene. 

Vi skriver nå at:    
* $P(H)=$ Sannsynligheten for $H$   
* $P(S)=$ Sannsynligheten for $S$

**(c)** Hvis $H$ og $S$ er komplementære, hva blir summen $P(H)+P(S)$?

**(d)** Kom med et forslag til hva sannsynlighetene $P(H)$ og $P(S)$ kan være, basert på de relative frekvensene som du har funnet tidligere. Stemmer dette med svaret i (c)?

**(Ekstra)** Hvis du tror at forslaget ditt i (d) er den riktige sannsynligheten, forklar hvorfor. 



## Ekstraoppgave: Endre simuleringsfunksjonen

Tenk deg at vi plukker et tilfeldig tall i listen $1,2,3,4,5,6,7,8,9,10$.
* Sannsynligheten for å plukke $1$ er $10\%$. 
* Sannsynligheten for å plukke et av tallene $1-3$ er $30\%$.
* Sannsynligheten for å plukke et av tallene $1-7$ er $70\%$. 
* Det samme mønsteret fortsetter. 
 

**(a)** Vi kan bruke funksjonen `randint(1,10)` for å få programmet til å plukke et tilfeldig tall i listen $1,2,3,4,5,6,7,8,9,10$. Sjekk at funksjonen fungerer ved å kjøre koden under flere ganger. 

In [ ]:
tilfeldig_tall = randint(1,10)
print(tilfeldig_tall)

**(b)** Husk nå at funksjonen `random_rooks()` lager et brett med tilfeldig tårnplassering, og at det er $50\%$ sannsynlighet for at tårn settes på et felt. Gå tilbake og endre på `random_rooks()` slik at sannsynligheten for at tårn settes på et felt er: 
* $30 \%$
* $70 \%$
* Velg eget prosenttall

**(c)** Undersøk hvordan svaret på de neste oppgavene påvirkes av endringen.

# Løsningsforslag

## Oppgave 1
**(b)**

In [ ]:
def random_rooks():
    
    #vi oppretter et tomt sjakkbrett:
    brett = Board(None)
    
    #vi bruker for-løkke for å gå gjennom alle 64 felter:
    for i in range(64):
        
        #vi er nå på felt nummer i
        #vi "kaster kron og mynt" for å avgjøre om felt nummer i skal ha tårn:
        tilfeldig_tall = randint(0,1)  
        
        #hvis vi fikk 1, skal vi sette tårn på felt nummer i:
        if tilfeldig_tall == 1:
            brett.set_piece_at(i, r)
            
        #hvis vi fikk 0, skal vi la feltet stå tomt, altså skal vi ikke gjøre noe
    
    #vi er ute av for-løkken og har en tilfeldig tårnplassering
    #vi returnerer sjakkbrettet
    return brett

## Oppgave 2

**(a)** Hver elev vil få ulike tabeller, så her finnes ingen fasit. 

**(b)** Relativ frekvens for hendelsen $H$, altså at hvit har en trygg kongevei, regnes ut med formelen: 

$$R(H) = \frac{\text{Antall kryss for hvit i tabellen}}{\text{Antall rader i tabellen}} $$

Vi multipliserer med 100 dersom vi ønsker svaret i prosent.

Relativ frekvens for hendelsen $S$, altså at svart har en trygg kongevei, regnes ut med tilsvarende formel:

$$R(S) = \frac{\text{Antall kryss for svart i tabellen}}{\text{Antall rader i tabellen}} $$

Igjen vil disse tallene variere for hver elev, men det forventes at både $R(H)$ og $R(S)$ kommer nærmere $0.5$ ($50\%$) jo flere rader vi har i tabellen. Det finnes en matematisk forklaring på dette som er gitt i kursheftet.

## Oppgave 3

**(a)** Det tar noen sekunder å kjøre koden.

In [ ]:
antall_simuleringer = 500  #antall simuleringer totalt
antall_hvit = 0 #antall simuleringer der hvit har trygg kongevei
antall_svart = 0 #antall simuleringer der svart har trygg kongevei

#vi bruker for-løkke for å simulere så mange ganger som skrevet ovenfor:
for i in range(antall_simuleringer):
    
    #vi er nå i simulering nummer i
    #vi lager et nytt brett med tilfeldig tårnplassering, ved å bruke funksjonen random_rooks():
    tilfeldig_brett = random_rooks()
    
    #vi spør programmet om hvem som har trygge kongeveier, og lagrer resultatene i variabler:
    liste = who_has_path(tilfeldig_brett)
    hvit_har_vei = liste[0]
    svart_har_vei = liste[1] 
    
    #variabelen hvit_har_vei er True hvis hvit har en trygg kongevei
    #variabelen svart_har_vei er True hvis svart har en trygg kongevei
    
    #sjekker om hvit har trygg kongevei:
    if (hvit_har_vei):
        
        #hvit har en trygg vei i denne simuleringen, og vi øker derfor antall_hvit med én
        antall_hvit = antall_hvit + 1 
    
    #sjekker om svart har trygg kongevei: 
    if (svart_har_vei):
        
        #svart har en trygg vei i denne simuleringen, og vi øker derfor antall_svart med én
        antall_svart = antall_svart + 1
        
    #vi har testet både hvit og svart i denne simuleringen, og kan gå til neste simulering i for-løkka
    
#vi har nå gjort alle simuleringene og kan regne ut de relative frekvensene:
relativ_frekvens_hvit = antall_hvit/antall_simuleringer
relativ_frekvens_svart = antall_svart/antall_simuleringer

#vi printer resultatene:
print(antall_simuleringer, "simuleringer")
print(antall_hvit, "vellykede simuleringer for hvit")
print(antall_svart, "vellykede simuleringer for svart")
print("R(H)=", relativ_frekvens_hvit)
print("R(S)=", relativ_frekvens_svart)

## Oppgave 4

**(a)** Vi gjør 50 simuleringer:

In [ ]:
for i in range(50):
    brett= random_rooks()
    liste = who_has_path(brett)
    print(liste)

**(a1)** Ingen av simuleringene ga resultatet `[False, False]`. Dette tyder på at det **ikke** finnes tårnbrett der hverken hvit eller svart har trygg kongevei. 

**(a2)** Ingen av simuleringene ga resultatet `[True, True]`. Dette tyder på at det **ikke** finnes tårnbrett der både hvit og svart har trygg kongevei. 

**(b)** Alle simuleringene gir resultatet `[True, False]` eller `[False, True]`. Dette tyder på at alle tårnbrett er i enten $H$ eller $S$, men ingen tårnbrett er i begge. Altså ser det ut til at $H$ og $S$ er komplementære hendelser. 

**(c)** Hvis $H$ og $S$ er komplementære, så kan vi bruke formelen $P(H) + P(S) = 1$.

**(d)** Det ser ut til at $P(H)=0.5$ og $P(S)=0.5$, altså at det er $50\%$ sannsynlighet fort begge hendelsene. Når vi setter dette inn i formelen ovenfor, så får vi at $0.5+0.5 = 1$, så resultatet stemmer med hypotesen om at $H$ og $S$ er komplementære.

**(Ekstra)** Vi skal forsøke å gi en matematisk forklaring på at det er $50\%$ sannsynlighet for begge hendelsene. Siden $P(H)$ og $P(S)$ er ukjent, setter vi: 

$P(H) = x$   
$P(S) = y$

Vi kan begynne med å si at $x=y$. En forklaring på dette er at hvits konge og svart konge faktisk har helt like problemer: 
* De skal komme seg fra en side av brettet til den andre.
* De kan gjøre de samme bevegelsene.
* De har lik sannsynlighet for å møte hindringer (fordi det er like stor sannsynlighet for at et felt er tomt som at det har tårn). 

Siden problemene er helt like, må altså hvit og svart konge ha like stor sannsynlighet for å komme seg over brettet, det vil si at $x=y$. (Men dette betyr **ikke** at hvit og svarts konge kommer seg over på de samme tårnbrettene. Det betyr bare at når vi kjører `random_rooks()`, så  er det like sannsynlig å få et tårnbrett der hvit kommer seg over, som å få et tårnbrett der svart kommer seg over.)

I oppgave (b) la vi fram en hypotese om at $H$ og $S$ er komplementære. Husk at vi ikke har noe bevis for dette, altså kan vi ikke være sikre på at det stemmer. Vi gjør likevel den antagelsen og vi kan da bruke formelen $x+y=1$. (I kursheftet vil du finne et matematisk forklaring på at $H$ og $S$ er komplementære).

Vi har nå to formler som sammen danner et likningssett:

$x=y$    
$x+y=1$    

Når vi løser dette ligningssettet kommer vi fram til at $x=0.5$ og $y=0.5$. 

## Ekstraoppgave

**(b)** Her kaller vi funksjonen `random_rooks2()` for å unngå å lage krøll for tidligere oppgaver.

In [ ]:
def random_rooks2():
    brett = Board(None)
    for i in range(64):
        
        #første endring: vi ber om et tilfeldig tall mellom 1 og 10
        tilfeldig_tall = randint(1,10)   
        
        #andre endring: 
        #for at det skal være 30% sannsynlighet for å plassere tårn,
        #må vi sjekke om tilfeldig_tall er mellom 1-3, og kun plassere tårn når svaret er ja
        
        if tilfeldig_tall <= 3:  #dette er True hvis tilfeldig_tall er mellom 1 og 3
            brett.set_piece_at(i, r)
        
    return brett

**(c)** Her finnes det faktisk ingen fasit! Siden sannsynligheten for tomt felt øker med $20\%$, skulle man kanskje tro at sannsynligheten for hvit kongevei øker med $20\%$ også. Men når du gjør simuleringer, vil du se at sannsynligheten ser ut til å gå opp med mye mer enn dette, til over $95\%$! Sannsynligheten for svart kongevei ser ut til å gå ned tilsvarende mye, til under $5\%$!

Hvis du prøver andre sannsynligheter for å plassere tårn, vil du se lignende mønster. Å senke sannsynligheten vil være svært gunstig for hvit konge, mens å øke sannsynligheten vil  være svært gunstig for svart konge. 

Å finne sannsynligheten for $H$ og $S$ matematisk er nå et nytt problem, siden vi har endret på det tilfeldige forsøket. Forskjellen er at vi har endret på sannsynligheten for å plassere tårn til $30\%$. Nå er ikke lenger alle utfall (tårnbrett) like sannsynlig. For eksempel er et utfall med 40 tomme felter nå mer sannsynlig enn et utfall med 40 tårn. Å finne de riktige sannsynlighetene nå er et interessant og vanskelig problem. Her kreves nok matte på høyere nivå (universitetsnivå)!